In [ ]:
# url = https://oauth.yandex.ru/authorize?response_type=token&client_id=Ваш client_id


import requests
import json
import os 
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
import calendar
from typing import Optional, Dict


load_dotenv()
token = os.getenv('token')

True

In [47]:
def fetch_phrase_dynamics_simple(phrase: str, period: str, from_date: Optional[str] = None, 
                               to_date: Optional[str] = None) -> pd.DataFrame:
    """
    Получает динамику запросов по фразе и возвращает DataFrame
    """
    url = "https://api.wordstat.yandex.net/v1/dynamics"

    payload = {
        "phrase": phrase,
        "period": period,
        "fromDate": from_date
    }
    
    if to_date:
        payload["toDate"] = to_date
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}'
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if 'dynamics' in data and data['dynamics']:
            df = pd.DataFrame(data['dynamics'])
            df['phrase'] = phrase  # Добавляем колонку с фразой
            return df
    
    return pd.DataFrame()




def get_info_ws(phrases: Dict
                ,period: str ="monthly"
                ,from_date: str = None
                ,to_date: Optional[str] = None) -> pd.DataFrame:
    """
    Приниматет словарь фраз и выводи информацию по году в динамике этих фраз
    """
    if from_date == None:
        from_date = datetime.now().replace(year=datetime.now().year-1, day=1).strftime("%Y-%m-%d")
    if to_date == None:
        current_date = datetime.now()
        last_day = calendar.monthrange(current_date.year, current_date.month)[1]
        to_date = current_date.replace(day=last_day).strftime("%Y-%m-%d")


    all_data = []
    
    for phrase in phrases:
        df = fetch_phrase_dynamics_simple(
            phrase=phrase,
            period=period,
            from_date=from_date,
            to_date=to_date
        )
        
        if not df.empty:
            all_data.append(df)
        else:
            print(f"Нет данных для фразы: {phrase}")
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        
        return combined_df
    else:
        return pd.DataFrame()




In [48]:
phrases = [
        "кот"
    ]

In [49]:
df = get_info_ws(
    phrases = phrases
)
df

,date,count,share,phrase
0,2024-11-01,14656375,0.126068,кот
1,2024-12-01,15681954,0.130701,кот
2,2025-01-01,16342861,0.137720,кот
3,2025-02-01,15280626,0.133910,кот
4,2025-03-01,15774864,0.131619,кот
5,2025-04-01,14022319,0.123068,кот
6,2025-05-01,14456552,0.135419,кот
7,2025-06-01,14448083,0.139470,кот
8,2025-07-01,14895614,0.140688,кот
9,2025-08-01,15280353,0.146811,кот
